In [ ]:
import numpy as np 
import cvxpy
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
X = np.array(Image.open("data/input_2.png").resize((32,32)))[:,:,:3]
ref = np.array(Image.open("data/input_2.png").resize((32,32)))[:,:,:3]

In [ ]:
ref[:,:] = ref[:,:]/2

In [ ]:
Image.fromarray(X)

In [ ]:
Image.fromarray(ref)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,:,0], X[:,:,1], X[:,:,2])

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(ref[:,:,0], ref[:,:,1], ref[:,:,2])

In [ ]:
X.shape

In [ ]:
X_h = X.shape[0]
X_w = X.shape[1]
ref_h = ref.shape[0]
ref_w = ref.shape[1]
X_dim = X_h*X_w
ref_dim = ref_h*ref_w

In [ ]:
X_dim

In [ ]:
ref_dim

In [ ]:
C = np.zeros((X_dim, ref_dim))

In [ ]:
X[0][0]

In [ ]:
ref[0][0]

In [ ]:
for i in range(X_dim):
    for j in range(ref_dim):
        C[i][j] = ((X[i//X_h][i%X_w] - ref[j//ref_h][j%ref_w])**2).sum()

In [ ]:
a = np.ones(X_dim)/X_dim
b = np.ones(ref_dim)/ref_dim

In [ ]:
eps = 0.2
K = np.exp(-C/eps)
u = np.ones(X_dim)

for i in range(1000):
    v = b / (K.T @ u)
    u = a / (K @ v)

f = eps*np.log(u + 1e-9)
g = eps*np.log(v + 1e-9)
    
P = u.reshape(X_dim,1) * K * v.reshape(1,ref_dim)

In [ ]:
# x = cvxpy.Variable((X_dim, ref_dim), nonneg=True)
# objective = cvxpy.sum(cvxpy.multiply(C,x))
# constraints = []
# constraints += [cvxpy.sum(x[i,:]) == a[i] for i in range(len(a))]
# constraints += [cvxpy.sum(x[:,j]) == b[j] for j in range(len(b))]
# constraints += [x>=0]
# prob = cvxpy.Problem(cvxpy.Minimize(objective), constraints)
# prob.solve()
# P = x.value

In [ ]:
P

In [ ]:
y = np.zeros((X_h, X_w,3))

In [ ]:
y.dtype

In [ ]:
for i in range(X_dim):
    a_i = np.zeros(3)
    for j in range(ref_dim):
        a_i += P[i][j]*ref[j//ref_h][j%ref_w]*len(b)
    y[i//X_h][i%X_w] = a_i

In [ ]:
y = y.astype(X.dtype)

In [ ]:
new = Image.fromarray(y)

In [ ]:
new.show()